### Quick Start

The `ssms` package serves two purposes. 

1. Easy access to *fast simulators of sequential sampling models*
   
2. Support infrastructure to construct training data for various approaches to likelihood / posterior amortization

We provide two minimal examples here to illustrate how to use each of the two capabilities.


#### Install 

Let's start with *installing* the `ssms` package.

You can do so by typing,

`pip install git+https://github.com/AlexanderFengler/ssm_simulators`

in your terminal.

Below you find a basic tutorial on how to use the package.

#### Tutorial

In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import ssms

#### Using the Simulators

Let's start with using the basic simulators. 
You access the main simulators through the  `ssms.basic_simulators.simulator` function.

To get an idea about the models included in `ssms`, use the `config` module.
The central dictionary with metadata about included models sits in `ssms.config.model_config`. 

In [2]:
# Check included models
list(ssms.config.model_config.keys())[:10]

['ddm',
 'ddm_legacy',
 'ddm_deadline',
 'angle',
 'weibull',
 'levy',
 'levy_angle',
 'full_ddm',
 'gamma_drift',
 'gamma_drift_angle']

In [3]:
# Take an example config for a given model
ssms.config.model_config["ddm"]

{'name': 'ddm',
 'params': ['v', 'a', 'z', 't'],
 'param_bounds': [[-3.0, 0.3, 0.1, 0.0], [3.0, 2.5, 0.9, 2.0]],
 'boundary': <function ssms.basic_simulators.boundary_functions.constant(t=0)>,
 'n_params': 4,
 'default_params': [0.0, 1.0, 0.5, 0.001],
 'hddm_include': ['z'],
 'nchoices': 2}

In [4]:
ssms.config.model_config["weibull"]["params"]

['v', 'a', 'z', 't', 'alpha', 'beta']

In [5]:
dict_out = ssms.basic_simulators.simulator._make_valid_dict({'v': 0.0, 'a': 1.65, 'z': 0.5, 't': 1.0005, 'alpha': 0.6, 'beta': 2.}) #, ssms.config.model_config["weibull"]["params"])
theta = ssms.basic_simulators.simulator._theta_dict_to_array(dict_out, ssms.config.model_config["weibull"]["params"])

t_s = np.arange(0, 20 + 0.001, 0.001).astype(np.float32)
boundary = np.zeros(t_s.shape, dtype = np.float32)
boundary[:] = theta[:, 1] * ssms.basic_simulators.boundary_functions.weibull_cdf(t_s, alpha=theta[:, 4], beta=theta[:, 5])

ssms.basic_simulators.simulator.cssm.ddm_flexbound(v = theta[:, 0],
                                                   a = theta[:, 1],
                                                   z = theta[:, 2],
                                                   t = theta[:, 3],
                                                   boundary_fun = ssms.basic_simulators.boundary_functions.weibull_cdf,
                                                   boundary_multiplicative=True,
                                                   boundary_params={"alpha": theta[:, 4], "beta": theta[:, 5]},
                                                   random_state = 42)

{'rts': array([[[1.2685001]],
 
        [[2.3635077]],
 
        [[2.1044955]],
 
        ...,
 
        [[1.6864947]],
 
        [[1.6514952]],
 
        [[1.3854985]]], dtype=float32),
 'choices': array([[[-1]],
 
        [[ 1]],
 
        [[-1]],
 
        ...,
 
        [[ 1]],
 
        [[ 1]],
 
        [[ 1]]], dtype=int32),
 'metadata': {'v': array([0.], dtype=float32),
  'a': array([1.65], dtype=float32),
  'z': array([0.5], dtype=float32),
  't': array([1.0005], dtype=float32),
  's': 1.0,
  'alpha': array([0.6], dtype=float32),
  'beta': array([2.], dtype=float32),
  'delta_t': 0.0010000000474974513,
  'max_t': 20.0,
  'n_samples': 20000,
  'simulator': 'ddm_flexbound',
  'boundary_fun_type': 'weibull_cdf',
  'possible_choices': [-1, 1],
  'trajectory': array([[ 0.0000000e+00],
         [ 2.8346935e-02],
         [-1.5303306e-04],
         ...,
         [-9.9900000e+02],
         [-9.9900000e+02],
         [-9.9900000e+02]], dtype=float32),
  'boundary': array([1.65      , 1

In [9]:
(theta[:, 1] * ssms.basic_simulators.boundary_functions.weibull_cdf(t_s, alpha=theta[:, 4], beta=theta[:, 5])).shape

(20001,)

In [17]:
ssms.basic_simulators.drift_functions.gamma_drift(t_s, shape = 2.0, scale = 0.5, c = 0.5).shape

(20001,)

In [14]:
help(ssms.basic_simulators.drift_functions.gamma_drift)

Help on function gamma_drift in module ssms.basic_simulators.drift_functions:

gamma_drift(t=array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9, 10. , 10.1, 10.2, 10.3, 10.4, 10.5, 10.6, 10.7, 10.8, 10.9,
       11. , 11.1, 11.2, 11.3, 11.4, 11.5, 11.6, 11.7, 11.8, 11.9, 12. ,
       12.1, 12.2, 12.3, 12.4, 12.5, 12.6, 12.7, 12.8, 12.9, 13. , 13.1,
       13.2, 13.3, 13.4, 13.5, 

In [26]:
theta.shape[0]

1

In [ ]:
np.ndarray[float, ndim = 1] v,
          np.ndarray[float, ndim = 1] a,
                  np.ndarray[float, ndim = 1] z,
                  np.ndarray[float, ndim = 1] t,
                  float s = 1,
                  float delta_t = 0.001,
                  float max_t = 20,
                  int n_samples = 20000,
                  int n_trials = 1,
                  boundary_fun = None, # function of t (and potentially other parameters) that takes in (t, *args)
                  boundary_multiplicative = True,
                  boundary_params = {},
                  random_state = None,
                  smooth = False,
                  

**Note:**
The usual structure of these models includes,

- Parameter names (`'params'`)
- Bounds on the parameters (`'param_bounds'`)
- A function that defines a boundary for the respective model (`'boundary'`)
- The number of parameters (`'n_params'`)
- Defaults for the parameters (`'default_params'`)
- The number of choices the process can produce (`'nchoices'`)

The `'hddm_include'` key concerns information useful for integration with the [hddm](https://github.com/hddm-devs/hddm) python package, which facilitates hierarchical bayesian inference for sequential sampling models. It is not important for the present tutorial.

In [6]:
from ssms.basic_simulators.simulator import simulator

sim_out = simulator(
    model="weibull", 
    theta={'v': 0.0, 'a': 1.65, 'z': 0.5, 't': 1.0005, 'alpha': 0.6, 'beta': 2.}, 
    n_samples=1000,
    smooth_unif = False)

{'v': 0.0, 'a': 1.65, 'z': 0.5, 't': 1.0005, 'alpha': 0.6, 'beta': 2.0}
theta after transform
[[0.     1.65   0.5    1.0005 0.6    2.    ]]
(1, 6)
made it here
before passed
passed
<class 'numpy.ndarray'>
(20001,)
[1.65]
{'alpha': 0.6, 'beta': 2.0}


TypeError: only length-1 arrays can be converted to Python scalars

The output of the simulator is a `dictionary` with three elements.

1. `rts` (array)
2. `choices` (array)
3. `metadata` (dictionary)

The `metadata` includes the named parameters, simulator settings, and more.

#### Using the Training Data Generators

The training data generators sit on top of the simulator function to turn raw simulations into usable training data for training machine learning algorithms aimed at posterior or likelihood armortization.

We will use the `data_generator` class from `ssms.dataset_generators`. Initializing the `data_generator` boils down to supplying two configuration dictionaries.

1. The `generator_config`, concerns choices as to what kind of training data one wants to generate.
2. The `model_config` concerns choices with respect to the underlying generative *sequential sampling model*. 

We will consider a basic example here, concerning data generation to prepare for training [LANs](https://elifesciences.org/articles/65074).

Let's start by peeking at an example `generator_config`.

In [5]:
ssms.config.data_generator_config["lan"]

{'output_folder': 'data/lan_mlp/',
 'dgp_list': 'ddm',
 'nbins': 0,
 'n_samples': 100000,
 'n_parameter_sets': 10000,
 'n_parameter_sets_rejected': 100,
 'n_training_samples_by_parameter_set': 1000,
 'max_t': 20.0,
 'delta_t': 0.001,
 'pickleprotocol': 4,
 'n_cpus': 'all',
 'kde_data_mixture_probabilities': [0.8, 0.1, 0.1],
 'simulation_filters': {'mode': 20,
  'choice_cnt': 0,
  'mean_rt': 17,
  'std': 0,
  'mode_cnt_rel': 0.95},
 'negative_rt_cutoff': -66.77497,
 'n_subruns': 10,
 'bin_pointwise': False,
 'separate_response_channels': False,
 'smooth_unif': True}

You usually have to make just few changes to this basic configuration dictionary.
An example below.

In [6]:
from copy import deepcopy

# Initialize the generator config (for MLP LANs)
generator_config = deepcopy(ssms.config.data_generator_config["lan"])
# Specify generative model (one from the list of included models mentioned above)
generator_config["dgp_list"] = "ddm"
# Specify number of parameter sets to simulate
generator_config["n_parameter_sets"] = 1000
# Specify how many samples a simulation run should entail
generator_config["n_samples"] = 2000
generator_config["n_cpus"] = 1
generator_config["smooth_unif"] = True

Now let's define our corresponding `model_config`.

In [7]:
model_config = ssms.config.model_config["ddm"]
print(model_config)

{'name': 'ddm', 'params': ['v', 'a', 'z', 't'], 'param_bounds': [[-3.0, 0.3, 0.1, 0.0], [3.0, 2.5, 0.9, 2.0]], 'boundary': <function constant at 0x11fe72710>, 'n_params': 4, 'default_params': [0.0, 1.0, 0.5, 0.001], 'hddm_include': ['z'], 'nchoices': 2}


We are now ready to initialize a `data_generator`, after which we can generate training data using the `generate_data_training_uniform` function, which will use the hypercube defined by our parameter bounds from the `model_config` to uniformly generate parameter sets and corresponding simulated datasets.

In [8]:
my_dataset_generator = ssms.dataset_generators.lan_mlp.data_generator(
    generator_config=generator_config, model_config=model_config
)

changed
checking:  data/lan_mlp/


In [23]:
def my_wrapper(random_seed_0 = [41, 42, 43], random_seed_1 = [41, 42, 43]):
    out = []
    for i,j in zip(random_seed_0, random_seed_1):
        np.random.seed(i)
        out.append(simulator(model="ddm", theta={"v": 0, "a": 1, "z": 0.5, "t": 0.5}, n_samples=1000,
        smooth_unif = True, random_state=j))
    return out

In [24]:
out = my_wrapper()

In [25]:
out

[{'rts': array([[4.12492   ],
         [2.1760223 ],
         [1.1929946 ],
         [2.6870239 ],
         [1.0399966 ],
         [1.4969907 ],
         [1.7720034 ],
         [1.9160101 ],
         [0.9559977 ],
         [1.245994  ],
         [0.94099784],
         [1.2259942 ],
         [0.9799974 ],
         [2.2430253 ],
         [0.6370001 ],
         [1.0929959 ],
         [2.891009  ],
         [0.8259994 ],
         [1.1049957 ],
         [0.8639989 ],
         [1.3559926 ],
         [2.4400346 ],
         [1.3589926 ],
         [1.1389954 ],
         [0.95499766],
         [1.4449914 ],
         [0.8269993 ],
         [1.0829961 ],
         [1.0699962 ],
         [1.1479952 ],
         [0.6140001 ],
         [1.2379941 ],
         [0.7670001 ],
         [1.7330016 ],
         [1.3909922 ],
         [1.0629964 ],
         [1.1529951 ],
         [1.2079945 ],
         [1.2029946 ],
         [1.2739936 ],
         [0.87699866],
         [0.9239981 ],
         [0.9819974 ],
    

In [9]:
training_data = my_dataset_generator.generate_data_training_uniform(save=False)

simulation round: 1  of 10
no multiprocessing
(238398742, 102270218)
running through smooth uni
(249450167, 323360142)
running through smooth uni
running through smooth uni
(9371619, 132810221)
running through smooth uni
(174006344, 186465682)
running through smooth uni
running through smooth uni
(36510354, 78227155)
running through smooth uni
(210128072, 322624310)
running through smooth uni
running through smooth uni
(234158102, 222747215)
running through smooth uni
(144335871, 124821367)
running through smooth uni
(150733225, 359918695)
running through smooth uni
running through smooth uni
(95307157, 397181670)
running through smooth uni
(73480002, 198827658)
running through smooth uni
(181544270, 313411769)
running through smooth uni
running through smooth uni
(168009606, 355203962)
running through smooth uni
(239201906, 85221927)
running through smooth uni
(12023480, 115854600)
running through smooth uni
running through smooth uni
(132438767, 38439259)
running through smooth uni
(

KeyboardInterrupt: 

`training_data` is a dictionary containing four keys:

1. `data` the features for [LANs](https://elifesciences.org/articles/65074), containing vectors of *model parameters*, as well as *rts* and *choices*.
2. `labels` which contain approximate likelihood values
3. `generator_config`, as defined above
4. `model_config`, as defined above

You can now use this training data for your purposes. If you want to train [LANs](https://elifesciences.org/articles/65074) yourself, you might find the [LANfactory](https://github.com/AlexanderFengler/LANfactory) package helpful.

You may also simply find the basic simulators provided with the **ssms** package useful, without any desire to use the outputs into training data for amortization purposes.

##### END